<a href="https://colab.research.google.com/github/majidahmadics/Machine-Learning-Projects/blob/main/RNN_for_Asset_Pricing/LSTM_For_Asset_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

In [47]:
# Initialize
ticker = "AAPL"
start_date = "2023-01-01"
end_date = "2024-01-01"

In [51]:
# Step 1: Data Collection
def fetch_stock_data(ticker, start_date, end_date):
  # Fetch the data
  stock_data = yf.download(ticker, start=start_date, end=end_date)
  return stock_data

# Step 2: Data Preprocessing
def preprocess_data(data, seq_length):
  # Normalize the data
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled_data = scaler.fit_transform(data.reshape(-1, 1))

  # Create sequences for training and testing
  sequences = []
  for i in range(len(scaled_data) - seq_length):
      sequence = scaled_data[i:i+seq_length]
      sequences.append(sequence)

  sequences = np.array(sequences)
  # Split the sequences into input features and target labels
  X = sequences[:, :-1]
  y = sequences[:, -1]

  # Split the data into training and testing sets
  train_size = int(len(sequences) * 0.80)
  X_train, X_test = X[:train_size], X[train_size:]
  y_train, y_test = y[:train_size], y[train_size:]

  return X_train, X_test, y_train, y_test, scaler

# Step 3: Model Building
def build_lstm_model(input_shape):
  model = Sequential()
  model.add(LSTM(units=50, input_shape=input_shape))
  model.add(Dense(units=1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

# Step 4: Model Training
def train_model(model, X_train, y_train, epochs, batch_size=32):
  model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Step 5: Model Evaluation
def evaluate_model(model, X_test, y_test, scaler):
  # Make predictions
  predictions = model.predict(X_test)
  predictions = scaler.inverse_transform(predictions)
  y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

  # Calculate RMSE
  rmse = np.sqrt(mean_squared_error(y_test, predictions))
  return rmse, predictions, y_test

In [53]:
# Main function
def main():
  # Step 1: Data Collection
  stock_data = fetch_stock_data(ticker, start_date, end_date)

  # Step 2: Data Preprocessing
  seq_length = 10
  X_train, X_test, y_train, y_test, scaler = preprocess_data(stock_data['Close'].values, seq_length)

  # Step 3: Model Building
  input_shape = (X_train.shape[1], X_train.shape[2])  # Shape of input for LSTM
  model = build_lstm_model(input_shape)

  # Step 4: Model Training
  train_model(model, X_train, y_train, epochs=500)

  # Step 5: Model Evaluation
  rmse, predictions, actual = evaluate_model(model, X_test, y_test, scaler)
  print("Test RMSE:", rmse)

  # Calculate accuracy between 0 and 1
  accuracy = 1 / (1 + rmse)  # Simple formula for converting RMSE to accuracy
  print("Accuracy:", accuracy)

  # Plot the actual and predicted data
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=stock_data.index[-len(predictions):], y=actual.flatten(), mode='lines', name='Actual'))
  fig.add_trace(go.Scatter(x=stock_data.index[-len(predictions):], y=predictions.flatten(), mode='lines', name='Predicted'))
  fig.update_layout(title='Actual vs. Predicted Stock Prices', xaxis_title='Date', yaxis_title='Price')
  fig.show()

if __name__ == "__main__":
  main()

[*********************100%%**********************]  1 of 1 completed


Epoch 1/500
6/6 [==============================] - 2s 6ms/step - loss: 0.2100
Epoch 2/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0464
Epoch 3/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0129
Epoch 4/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0188
Epoch 5/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0041
Epoch 6/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0061
Epoch 7/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0054
Epoch 8/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0031
Epoch 9/500
6/6 [==============================] - 0s 7ms/step - loss: 0.0036
Epoch 10/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0032
Epoch 11/500
6/6 [==============================] - 0s 5ms/step - loss: 0.0030
Epoch 12/500
6/6 [==============================] - 0s 6ms/step - loss: 0.0030
Epoch 13/500
6/6 [==============================] - 0s 6ms/st